In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from collections import  defaultdict

# clusters = ['earth', 'saturn', 'uranus', 'venus']
# schedulers = ['ours', 'ff', 'wf', 'bf', 'csa']

clusters = ['earth', 'saturn', 'uranus', 'venus']
schedulers = ['ours', 'ff', 'wf', 'bf', 'csa']

results = defaultdict(lambda: defaultdict(float))
for cluster in clusters:
    for sch in schedulers:
        with open(f'./experiments/{cluster}/{sch}.json', 'r') as f:
            data = json.load(f)

        local_rate = []
        for job in data:
            if len(job['location']) == 0:
                print(cluster, sch, job)
                continue
            chunks = np.array_split(job['chunks'], len(job['location']))
            for i, worker in enumerate(job['location']):
                worker_chunks = chunks[i]
                local = [chunk['Location'] == worker for chunk in worker_chunks]
                if len(local) == 0:
                    continue
                local_rate.append(100 * sum(local) / len(worker_chunks))
        
        results[cluster][sch] = np.mean(local_rate)

In [11]:
results = pd.DataFrame(results).round(2).T
results.to_csv("result.csv", index=False)

In [14]:
results

,ours,ff,wf,bf,csa
earth,92.51,79.89,79.41,79.60,80.01
saturn,90.31,83.03,82.31,82.77,83.08
uranus,92.85,84.12,83.87,83.95,84.14
venus,92.42,80.00,79.35,79.57,80.20


In [16]:
results.mean()

ours    92.0225
ff      81.7600
wf      81.2350
bf      81.4725
csa     81.8575
dtype: float64

In [15]:
for col in results.columns[1:]:
    imp = 100 * (results['ours'] - results[col])/results[col]
    print(max(imp))

15.796720490674684
16.49666288880495
16.218592964824136
15.62304711911011
